## Проект 3. О вкусной и здоровой пище

In [147]:
import re
import numpy as np
import pandas as pd
from datetime import datetime as dt
import collections

In [148]:
data = pd.read_csv('main_task.csv')

In [149]:
print(data.columns)
data.describe()

Index(['Restaurant_id', 'City', 'Cuisine Style', 'Ranking', 'Rating',
       'Price Range', 'Number of Reviews', 'Reviews', 'URL_TA', 'ID_TA'],
      dtype='object')


,Ranking,Rating,Number of Reviews
count,40000.000000,40000.000000,37457.000000
mean,3676.028525,3.993037,124.825480
std,3708.749567,0.668417,295.666352
min,1.000000,1.000000,2.000000
25%,973.000000,3.500000,9.000000
50%,2285.000000,4.000000,33.000000
75%,5260.000000,4.500000,115.000000
max,16444.000000,5.000000,9660.000000


In [150]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [151]:
# data['City'].value_counts()
data['Cuisines'] = data['Cuisine Style'].apply(lambda x: len(x.strip('[]').split(', ')) if x is not np.nan else 1)

In [152]:
date_ptn = re.compile('\'\d+/\d+/\d+\'?')
data['Review dates'] = data['Reviews'].apply(lambda x: [dt.strptime(item, "'%m/%d/%Y'") for item in date_ptn.findall(x)])
data['Review DD'] = data['Review dates'].apply(lambda x: (x[0] - x[1]).days if len(x) >1 else 0)

In [161]:
data[['City', 'Ranking', 'Rating','Price Range', 'Number of Reviews']].head(50)


,City,Ranking,Rating,Price Range,Number of Reviews
0,Paris,5570.0,3.5,$$ - $$$,194.0
1,Stockholm,1537.0,4.0,NaN,10.0
2,London,353.0,4.5,$$$$,688.0
3,Berlin,3458.0,5.0,NaN,3.0
4,Munich,621.0,4.0,$$ - $$$,84.0
5,Oporto,1419.0,3.0,NaN,2.0
6,Milan,1722.0,4.0,$,50.0
7,Bratislava,826.0,3.0,NaN,9.0
8,Vienna,2692.0,4.0,NaN,NaN
9,Rome,4210.0,4.0,$,55.0


In [2]:
def split_string(string):
    return string.replace('[]', '').replace("'", '').split(', ')

cuisine_styles = data[data['Cuisines'] == data['Cuisines'].max()]['Cuisine Style'].values[0].strip("[]").split(', ')
selected_cuisines = collections.Counter()
cuisine_series = data[data['Cuisine Style'].notna()]['Cuisine Style']
for row in cuisine_series:
    for cuisine in split_string(row):
        selected_cuisines[cuisine] += 1

selected_cuisines = {'Cuisine ' + k: v  for k, v in sorted(selected_cuisines.items(), key=lambda item: item[1])}
selected_cuisines = dict(filter(lambda x: True if x[1] > 500 else False, selected_cuisines.items()))

NameError: name 'data' is not defined

In [155]:
def assign_values(values):
    existing = selected_cuisines.copy()

    for key, val in existing.items():
        existing[key] = 1 if values is not np.nan and key[8:] in split_string(values) else 0

    return list(existing.values())

# cuisines = pd.DataFrame(columns=['Restaurant_id'] + list(selected_cuisines.keys()))
# for row in data[['Restaurant_id', 'Cuisine Style']].loc:
#     cuisines.loc[row.name] = [row['Restaurant_id']] + assign_values(row['Cuisine Style'])
# cuisines.to_csv('cuisines.csv')

In [156]:
ratings = pd.read_csv('data_loaded.csv', index_col=[0])
cuisines = pd.read_csv('cuisines.csv', index_col=[0])

In [157]:
data_ratings = data.merge(ratings, on=data.index)

In [158]:
data_ratings.drop(['Restaurant_id_x', 'Restaurant_id_y', 'key_0'], axis=1, inplace=True)
full_data = data_ratings.merge(cuisines, on=data_ratings.index)

In [159]:
full_data.drop('key_0', axis=1, inplace=True)

In [160]:
full_data.to_csv('full.data.csv')